In [14]:
from sqlalchemy import *
engine = create_engine('sqlite:///onto.db', echo = False)
c = engine.connect()

In [70]:
c = engine.connect()
results = c.execute("""SELECT * FROM class_relations""")
results.fetchall()

[(1, 'hypernym', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#TandooriPizza', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#IndianPizza', 1),
 (2, 'concept', "https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Domino'SPizza", 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', 1),
 (3, 'isToppingOf', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Marinara', 1),
 (4, 'hyponym', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Pizza', 'https://serc.iiit.ac.in/downloads/ontology/pizza.owl#Peri-Peri', 1),
 (5, 'hypernym', 'https://serc.iiit.ac.in/downloads/ontology/securityontology.owl#Authentication', 'https://serc.iiit.ac.in/downloads/ontology/securityontology.owl#Authorization', 2),
 (6, 'hypernym', 'https://serc.iiit.ac.in/downloads/ontology/securityontology.owl#Firewall', 'https://serc.iiit.ac.in/downloads/ontology/securityontology.owl#NetworkSecurity', 2),
 (7, 'hyponym', 'https://s

In [21]:
engine = create_engine('sqlite:///onto.db', echo = True)
    # conn = sqlite3.connect('onto.db')
    # c = conn.cursor()
c = engine.connect()
trans = c.begin()
query = """SELECT * FROM node_decisions INNER JOIN nodes ON node_decisions.node_id =nodes.id """
result = c.execute(query)
result.fetchall()

2020-11-06 14:18:08,109 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-06 14:18:08,113 INFO sqlalchemy.engine.base.Engine ()
2020-11-06 14:18:08,119 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-06 14:18:08,130 INFO sqlalchemy.engine.base.Engine ()
2020-11-06 14:18:08,139 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-06 14:18:08,145 INFO sqlalchemy.engine.base.Engine SELECT * FROM node_decisions INNER JOIN nodes ON node_decisions.node_id =nodes.id 
2020-11-06 14:18:08,157 INFO sqlalchemy.engine.base.Engine ()


[(1, 4, 0, 12345678, 4, 1, 'https://serc.iiit.ac.in/downloads/ontology/test.owl#IndianPizza'),
 (2, 3, 0, 12345678, 3, 1, 'https://serc.iiit.ac.in/downloads/ontology/test.owl#TandooriPizza'),
 (3, 5, 0, 12345678, 5, 1, 'https://serc.iiit.ac.in/downloads/ontology/test.owl#spices')]

In [24]:
query = """SELECT * FROM ontologies"""
result = c.execute(query)
result.fetchall()

2020-11-06 14:40:10,360 INFO sqlalchemy.engine.base.Engine SELECT * FROM ontologies
2020-11-06 14:40:10,368 INFO sqlalchemy.engine.base.Engine ()


[(1, 'pizza', 0), (2, 'pizza', 1), (3, 'pizza', 1), (4, 'pizza', 1)]

In [31]:
from owlready2 import *

from re import finditer
def split_by_camel_case(identifier):
    # Split string by camel-case
    matches = finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0) for m in matches])

ont = get_ontology("data/input/ontologies/pizza.owl").load()
ont_classes = list(ont.classes())
classes = [split_by_camel_case(elem.label.first()).lower() for elem in ont_classes]

##if term1 not in set(classes) or term2 in set(classes):
    ##append it
# idx1 = classes.index(term1) 
# idx2 = classes.index(term2)
# ont_classes[idx1], ont_classes[idx2]

In [65]:
import itertools
from xml.dom import minidom
from urllib.request import urlopen

flatten = lambda l: [item for sublist in l for item in sublist]

# Define class to parse ontology
class Ontology():
    def __init__(self, ontology):
        '''
        Instantiates an Ontology object. 
        Args:
            ontology: Path to ontology file
        Returns:
            Parsed Ontology object
        '''
        self.ontology = ontology
        if ontology.startswith("https://") or ontology.startswith("http://"):
            self.ontology_obj = minidom.parse(urlopen(ontology + "/"))
        else:
            self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement # root node

        self.construct_mapping_dict()
        
        self.subclasses = self.parse_subclasses()
        self.classes = self.parse_classes()        
        self.instances = self.parse_instances()

    def construct_mapping_dict(self):
        '''
        Constructs ID to label mapping dict for ontologies where 
        entities are identified by IDs.
        '''
        elements = self.root.getElementsByTagName("owl:Class") + self.root.getElementsByTagName("owl:NamedIndividual") + self.root.getElementsByTagName("owl:Thing")
        mapping_dict = {self.extract_ID(el, False): self.return_label(el) for el in elements if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict = {elem: mapping_dict[elem] for elem in mapping_dict if mapping_dict[elem]}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return

    def parse_classes(self):
        '''
        Parse only classes
        '''
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_instances(self):
        '''
        Parses ontology to obtain (instance, concept) tuples
        Returns:
            list of instance-concept 2-tuples of the form (a,b)
        '''
        instances = self.root.getElementsByTagName("owl:NamedIndividual") + self.root.getElementsByTagName("owl:Thing")
        instance_pairs = []
        for instance in instances:
            concept = self.get_child_node(instance, "rdf:type")
            if concept:
                instance_pairs.append((self.extract_ID(concept[-1]), self.extract_ID(instance)))
        return list(set(self.filter_null(instance_pairs)))

    def return_label(self, el):
        '''
        Returns label of an element, and also detects language of the label
        '''
        label_element = [l for l in self.get_child_node(el, "rdfs:label") if l.getAttribute("xml:lang") == "en"]
        if not label_element:
            return ''
        label_element = label_element[0]
        return label_element.firstChild.nodeValue

    def get_child_node(self, element, tag):
        '''
        Returns child node with a specific tag name given DOM element 
        Args:
            element: DOM parent element
            tag: Name of tag of child
        Returns:
            DOM child element 
        '''
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]

    def parse_subclasses(self):
        '''
        Parses ontology to obtain subclass 2-tuples
        Returns:
            list of subclass 2-tuples of the form (a,b)
        '''
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                # Subclass of with inline IDs
                subclass_pairs.append((el, el.parentNode))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if level1_class:
                    if self.extract_ID(level1_class[0]):
                        # Subclass label under a level 1 tag
                        subclass_pairs.append((level1_class[0], el.parentNode))
                    else:
                        continue
        subclasses = [(self.extract_ID(a), self.extract_ID(b)) for (a,b) in subclass_pairs]
        subclasses = [elem for elem in subclasses if elem[0]!="Thing" and elem[1]!="Thing"]
        return subclasses

    def filter_null(self, data):
        return [el for el in data if el]

    def extract_ID(self, element, extract_label=True):
        '''
        Returns ID for a parsed DOM element. In ontologies where classes are represented by 
        numerical IDs, it returns the label (stored in mapping_dict)
        '''
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1].split(";")[-1]
        if extract_label and element_id in self.mapping_dict:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")

Ontology("data/input/ontologies/security.owl").instances

[('WKOITSicherheitshandbuch', '6. Network Security'),
 ('Exploitation_Degree_Relation', 'Exploitation_Degree_Relation_27'),
 ('ISO27001', 'A.6.2.3 Addressing security in third party agreements'),
 ('TopLevelThreat', 'Financial Loss'),
 ('Control', 'Water Alarm System Control'),
 ('ISO27001', 'A.8.1.2 Screening'),
 ('LowLevelThreat', 'Increasing Recovery Time'),
 ('ISO27001', 'A.12.4.1 Control of operational software'),
 ('WKOITSicherheitshandbuch',
  '5.8.2 Virus Avoidance and Recogniton by the User'),
 ('ISO27001', 'A.8.3.3 Removal of access rights'),
 ('ISO27001', 'A.6.1.1 Management commitment to information security'),
 ('ISO27001', 'A.10.8.4 Electronic messaging'),
 ('Level', 'Second Floor'),
 ('WKOITSicherheitshandbuch', '4. Personnel Measures'),
 ('Data Backup Policy B', 'DataBackupPolicyBMyOrganization'),
 ('ISO27001', 'A.10.6.2 Security of network services'),
 ('ISO27001', 'A.9.1.2 Physical entry controls'),
 ('Exploitation_Degree_Relation', 'Exploitation_Degree_Relation_17'),